In [ ]:
import scared

## Leakage Functions

In [ ]:
import scared
@scared.attack_selection_function
def key_pt_xor(plaintext, guesses):
    res = np.empty((plaintext.shape[0], len(guesses), plaintext.shape[1]), dtype='uint8')
    for i, guess in enumerate(guesses):
        res[:,i,:] = np.bitwise_xor(plaintext, guess)
    return res

@scared.attack_selection_function
def sbox_hd(plaintext, guesses):
    res = np.empty((plaintext.shape[0], len(guesses), plaintext.shape[1]), dtype='uint8')
    for i, guess in enumerate(guesses):
        res[:,i,:] = np.bitwise_xor(scared.aes.sub_bytes(np.bitwise_xor(plaintext, guess)), np.bitwise_xor(plaintext, guess))
    return res

@scared.attack_selection_function
def sbox_hw(plaintext, guesses):
    res = np.empty((plaintext.shape[0], len(guesses), plaintext.shape[1]), dtype='uint8')
    for i, guess in enumerate(guesses):
        res[:,i,:] = scared.aes.sub_bytes(np.bitwise_xor(plaintext, guess))
    return res



## T_Table

In [ ]:
from scared.models import HammingWeight
@numba.vectorize([numba.uint32(numba.uint8)])
def t_table_vec(x):
    return _T_TABLE_LUT[x]

class TTable(HammingWeight):
    def _compute(self, data, axis):
        if data.dtype.kind != 'u':
            raise ValueError(f'HammingWeight should take unsigned integer data as input, not {data.dtype}).')

        if data.dtype != self.expected_dtype:
            raise ValueError(f'Expected dtype for HammingWeight input data is {self.expected_dtype}, not {data.dtype}.')

        if data.shape[axis] < self.nb_words:
            raise ValueError(f'data should have at least {self.nb_words} as dimension with index {axis}, not {data.shape[axis]}.')

        result_data = t_table_vec(data)
        if self.nb_words > 1:
            final_w_dimension = data.shape[axis] // self.nb_words
            final_shape = [d if i != axis else final_w_dimension for i, d in enumerate(data.shape)]
            result = _np.zeros(final_shape, dtype='uint32').swapaxes(0, axis)
            result_data = _np.swapaxes(result_data, 0, axis)
            for i in range(result.shape[0]):
                slices = result_data[i * self.nb_words: (i + 1) * self.nb_words]
                result[i] = _np.sum(slices, axis=0)
            result_data = result
            result_data = _np.swapaxes(result, 0, axis)

        return result_data

## CW Project to estraces

In [ ]:
import estraces
#del cw_traces
cw_traces = estraces.read_ths_from_ram(np.array(project.waves), 
                                      plaintext=np.array(project.textins),
                                      ciphertext=np.array(project.textouts))

## Attack Object

In [ ]:
container = scared.Container(cw_traces)
a = scared.CPAAttack(selection_function=sbox_hd,
                    model=HammingWeight(),
                    discriminant=scared.maxabs)

## Running Attack and Printing Results

In [ ]:
%%time
a.run(container)
for byte in np.argmax(a.scores, axis=0):
    print("{:02X}".format(byte%0xFF))

## Print Results and Plot Correlation

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure()
key_guess = []
for i in range(16):
    results = a.results
    print("Correct key: {:02X} (corr= {})".format(project.keys[0][i], a.scores[project.keys[0][i]][i]))
    sorted_scores = np.argsort(a.scores, axis=0)[::-1]
    j=0
    pge = 0
    for score in sorted_scores:
        #print(hex(score[i]), a.scores[score[i]][i])
        if score[i] == project.keys[0][i]:
            pge = j 
        j += 1
    print("PGE={}".format(pge))
    guess = sorted_scores[0][i]
    sec_guess = sorted_scores[1][i]
    third_guess = sorted_scores[2][i]
    xrange = range(len(results[guess][i]))
    print("Best Guess is {:02X} ({:02X}) (Corr = {})".format(guess, project.keys[0][i], a.scores[guess][i]))
    #print("Second Guess is {:02X} ({:02X}) (Corr = {})".format(sec_guess, project.keys[0][i], a.scores[sec_guess][i]))
    #print("Third Guess is {:02X} ({:02X}) (Corr = {})".format(third_guess, project.keys[0][i], a.scores[third_guess][i]))
    p.line(xrange, results[guess][i], color="red")
    #p.line(xrange, results[sec_guess][i], color="green")
    #p.line(xrange, results[third_guess][i], color="green")
    p.line(xrange, results[project.keys[0][i]][i], color="green")
    key_guess.append(guess)
    
show(p)